In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
# -- years being used for evaluation
Year = '2004'

# -- use Fiscal or Calendar year
UseFiscalYear = True

# -- input path
InputPath = '../../shapeData/Tables/ParkQualityScores/'

# -- output path
WriteOutputPath = '../../shapeData/Tables/ZipcodeAllAttributes/'

In [3]:
# -- Bring in the socioeconomic Data
ZipcodesocioEconomicDataPath = os.path.join('../../shapeData/externalData/','ZipCodeExternalDataSummary.csv')
ZipcodesocioEconomicData = pd.read_csv(ZipcodesocioEconomicDataPath)

In [4]:
# -- Bring in the market Value Data
medianMarketValueZipCodePath=('../../shapeData/Tables/medianMarketValueZipCode.csv')
medianMarketValueZipCode = pd.read_csv(medianMarketValueZipCodePath)
medianMarketValueZipCode = medianMarketValueZipCode[['ZipCode','Market_Val']]

In [5]:
# -- Bring in the economic Data
economicDataZipCodePath=('../../shapeData/externalData/zbp13totals.csv')
economicDataZipCode = pd.read_csv(economicDataZipCodePath)

In [6]:
# -- Clean the economic Data and rename columns with better titles
economicDataZipCode = economicDataZipCode[['zip','emp','ap','est']]
economicDataZipCode = economicDataZipCode.rename(columns={'zip':'ZipCode','emp':'TotalMidMarchEmployees', 'ap':'TotalAnnualPayroll($1,000)', 'est':'TotalNumberofEstablishments'})

In [7]:
# -- Bring in Park Quality Data
if UseFiscalYear:
    parkQualityZipCode = pd.read_csv(InputPath + 'ParkQuality_ZIPCODE_%s_FiscalYr_07-13-2015.csv' % (Year), index_col=0)
else:
    parkQualityZipCode = pd.read_csv(InputPath + 'ParkQuality_ZIPCODE_%s_CalendarYr_07-13-2015.csv' % (Year), index_col=0)

# parkQualityZipCodePath=('../../shapeData/Tables/AmenityAreaParkQualityZIPCODE_2014_07-09-2015.csv')
# parkQualityZipCode = pd.read_csv(parkQualityZipCodePath)
# # -- Get rid of the duplicated index column
# del parkQualityZipCode['Unnamed: 0']

In [26]:
# -- Bring in Functional Parkland Data
ZipcodeFunctionalParklandPath=('/scratch/share/gdobler/parqa/output/Tables/CUSP_Adjusted_Spatial_Data.xlsx')
ZipcodeFunctionalParkland = pd.read_excel(ZipcodeFunctionalParklandPath)
ZipcodeFunctionalParklandSum = FunctionalParkland.groupby(by=['ZIPCODE'])['Acres_Adj'].sum().reset_index()

In [31]:
# -- Join the market value, socioeconomic, econoimic, functional parkland, and park-qality tables
ZipCodeMarketValueSocioeconomic = pd.merge(ZipcodesocioEconomicData, medianMarketValueZipCode, right_on='ZipCode', left_on= 'ZipCode5', how='left')
ZipcodeExternalData = pd.merge(ZipCodeMarketValueSocioeconomic, economicDataZipCode, on= 'ZipCode', how='left')
ZipCodeAllExternalData = pd.merge(parkQualityZipCode, ZipcodeExternalData, left_on='ZIPCODE', right_on = 'ZipCode', how='left')
ZipCodeAllAttributes = pd.merge(ZipCodeAllExternalData, ZipcodeFunctionalParklandSum, on = 'ZIPCODE', how= 'left')
ZipCodeAllAttributes['PopulationPerParkland'] = ZipCodeAllAttributes['TotalPopulation']/ZipCodeAllAttributes['Acres_Adj']
ZipCodeAllAttributes['EmployeesPerParkland'] = ZipCodeAllAttributes['TotalMidMarchEmployees']/ZipCodeAllAttributes['Acres_Adj']

In [ ]:
# -- Write to csv
now = datetime.now()
if UseFiscalYear:
    YearType = 'FiscalYr'
else:
    YearType = 'CalendarYr'
ZipCodeAllAttributes.to_csv(WriteOutputPath + 'AllAttributes_Zipcode_%s_%s_%s.csv' % ( Year.replace(',','&'), YearType, now.strftime('%m-%d-%Y')), sep=',')